## Яндекс Практикум, курс "Инженер Машинного Обучения" (2024 г.)
## Проект 3-го спринта: "Релиз модели в продакшн"
***
### Ноутбук для подготовки модели

Можно либо загрузить модель из MLflow, либо заново обучить с учетом результатов проекта 2-го спринта.
В данном ноутбуке мы используем 2-й способ и обучаем модель на всех сгенерированных признаках без дальнейшего их отбора с использованием ранее найденных наилучших параметров регрессионной модели, т.к. этому случаю соответствует наименьшая ошибка MAPE.

In [1]:
# Подключаем необходимые библиотеки

import os
from dotenv import load_dotenv

import pandas as pd
pd.options.display.max_columns = 100
pd.options.display.max_rows = 64
import numpy as np
import joblib

from sqlalchemy import create_engine
from datetime import datetime

from category_encoders import CatBoostEncoder
from catboost import CatBoostRegressor

from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    StandardScaler,
    OneHotEncoder, 
    QuantileTransformer, 
    PolynomialFeatures
)

from autofeat import AutoFeatRegressor

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Инциализируем генератор случайных чисел
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

Скачиваем исходные данные с квартирами, очищенные на предыдущих спринтах

In [4]:
try:
    # Скачиваем из локального файла, если ранее сохраняли
    data = pd.read_csv("../services/ml_service/data/clean_data.csv")

except: # Скачиваем из БД
    # Загружаем переменные окружения
    load_dotenv()

    # Создаем соединение
    username = os.environ.get('DB_DESTINATION_USER')
    password = os.environ.get('DB_DESTINATION_PASSWORD')
    host = os.environ.get('DB_DESTINATION_HOST')
    port = os.environ.get('DB_DESTINATION_PORT')
    db = os.environ.get('DB_DESTINATION_NAME')
    conn = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{db}', connect_args={'sslmode':'require'})

    # Выполняем SQL-запрос
    data = pd.read_sql('select * from clean_flats_dataset', conn, index_col='flat_id')

    # Сохраняем данные локально
    os.makedirs('../services/ml_service/data', exist_ok=True)
    data.to_csv('../services/ml_service/data/clean_data.csv')

In [5]:
data.head()

,id,floor,kitchen_area,living_area,rooms,is_apartment,studio,total_area,price,build_year,building_type_int,latitude,longitude,ceiling_height,flats_count,floors_total,has_elevator
flat_id,,,,,,,,,,,,,,,,,
8348,23114,8,10.6,56.000000,3,False,False,88.599998,10990000,2018,4,55.542187,37.483067,2.64,409,18,True
8350,23116,3,7.0,28.000000,2,False,False,44.700001,8999000,1967,4,55.857765,37.422684,2.64,143,9,True
8351,23118,16,10.9,54.799999,4,False,False,89.099998,24000000,1996,4,55.562908,37.570431,2.70,164,16,True
8352,23120,2,7.4,66.300003,4,False,False,93.000000,17500000,1965,1,55.653507,37.649426,2.70,59,6,True
8354,23122,4,9.1,17.700001,1,False,False,34.000000,7500000,1964,1,55.796406,37.459873,3.00,72,9,True


Выполняем дополнительную предобработку данных для обучения модели

In [6]:
# Удаляем выбросы цен
threshold = 1.5
Q1 = data['price'].quantile(0.25)
Q3 = data['price'].quantile(0.75)
IQR = Q3 - Q1
margin = threshold * IQR
lower = Q1 - margin
upper = Q3 + margin
data = data[data.price.between(lower, upper)]

# Вместо года постройки добавляем возраст здания
data['building_age'] = (datetime.now().year - data['build_year']).astype('float')

# Удаляем лишние колонки (studio является константным признаком)
data.drop(
    columns=['id', 'build_year', 'studio'], 
    inplace=True
)

# Изменяем тип количественных целых колонок на float, за исключением building_type_int, 
# который будет категориальным
int_col_names = data.select_dtypes('int').columns.drop('building_type_int')
data[int_col_names] = data[int_col_names].astype('float') 

# Изменяем тип булевских колонок на int, чтобы не использовать для них one hot encoding
bool_col_names = data.select_dtypes('bool').columns
data[bool_col_names] = data[bool_col_names].astype('int') 

Разделяем данные на обучающую и тестовую выборки

In [7]:
X_train, X_test, y_train, y_test = train_test_split(data[data.columns.drop('price')], 
                                                    data['price'],
                                                    test_size=0.2, 
                                                    random_state=RANDOM_STATE
                                                   )

Группируем признаки по типам

In [8]:
cat_features = X_train.select_dtypes(include=['object'])
is_bin_cat_features = cat_features.nunique() == 2
bin_cat_features = cat_features[is_bin_cat_features[is_bin_cat_features].index]
other_cat_features = cat_features[is_bin_cat_features[~is_bin_cat_features].index]
num_features = X_train.select_dtypes(include=['float']) 

Создаем энкодеры для кодирования существующих и генерации новых признаков

In [9]:
# Для бинарных категориальных признаков
encoder_oh = OneHotEncoder(
    categories='auto',
    handle_unknown='ignore', 
    sparse_output=False,
    drop='if_binary'
) 

# Для небинарных категориальных признаков
encoder_cb = CatBoostEncoder(random_state=RANDOM_STATE)

# Для генерации новых числовых признаков
encoder_pol = PolynomialFeatures(
    degree=2,
    include_bias=False
)

# Для генерации новых числовых признаков
encoder_q = QuantileTransformer(
    n_quantiles=100,
    random_state=RANDOM_STATE
)

# Для генерации новых числовых признаков: применяем к каждому числовому признаку функцию log(1+x) один раз
encoder_afr = AutoFeatRegressor(
    transformations=('1+', 'log'),
    feateng_steps=1,
    n_jobs=-1
)

Создаем трансформер данных

In [10]:
# Трансформер для преобразования числовых признаков (вкл. autofeat).
# NB: на выходе будут в т.ч. исходные числовые признаки, они соответствуют степени 1 полиномиального энкодера
num_preproc_w_afr = ColumnTransformer(
    [
        ('num_pol', encoder_pol, num_features.columns.tolist()),
        ('num_q', encoder_q, num_features.columns.tolist()),
        ('num_afr', encoder_afr, num_features.columns.tolist()) 
    ],
    remainder='drop',
    verbose_feature_names_out=True,
    n_jobs=-1
)

# Добавляем к трансформеру числовых признаков масштабирование
num_pipeline_w_afr = Pipeline(
    [
        ('num_preproc_w_afr', num_preproc_w_afr),
        ('num_sc', StandardScaler())
    ]
)

# Нормализуем числовые признаки после их преобразования.
preproc_w_afr = ColumnTransformer(
    [
        ('bin_cat', encoder_oh, bin_cat_features.columns.tolist()),
        ('other_cat', encoder_cb, other_cat_features.columns.tolist()),
        ('num_pipeline_w_afr', num_pipeline_w_afr, num_features.columns.tolist())
        
    ],
    remainder='passthrough',
    verbose_feature_names_out=True,
    n_jobs=-1
)

Создаем пайплайн, параметры для регрессионной модели берем из проекта 2-го спринта

In [11]:
best_params = {
    'learning_rate': 0.1, 
    'l2_leaf_reg': 0.1, 
    'depth': 7
}

model = CatBoostRegressor(
    loss_function='MAPE', 
    verbose=False, 
    random_state=RANDOM_STATE,
    **best_params
)
  
pipeline = Pipeline(
    [
        ('preproc_w_afr', preproc_w_afr),
        ('model', model)
    ]
)

In [12]:
# Обучаем пайплайн
print('Идет обучение пайплайна...')
%time pipeline.fit(X_train, y_train)

Идет обучение пайплайна...
CPU times: total: 4min 57s
Wall time: 1min


Pipeline(steps=[('preproc_w_afr',
                 ColumnTransformer(n_jobs=-1, remainder='passthrough',
                                   transformers=[('bin_cat',
                                                  OneHotEncoder(drop='if_binary',
                                                                handle_unknown='ignore',
                                                                sparse_output=False),
                                                  []),
                                                 ('other_cat',
                                                  CatBoostEncoder(random_state=42),
                                                  []),
                                                 ('num_pipeline_w_afr',
                                                  Pipeline(steps=[('num_preproc_w_afr',
                                                                   ColumnTransformer(n_jobs=-1,
                                                                                     transformers=[('num_pol...
                                                                                                     'total_area',
                                                                                                     'latitude',
                                                                                                     'longitude',
                                                                                                     'ceiling_height',
                                                                                                     'flats_count',
                                                                                                     'floors_total',
                                                                                                     'building_age'])])),
                                                                  ('num_sc',
                                                                   StandardScaler())]),
                                                  ['floor', 'kitchen_area',
                                                   'living_area', 'rooms',
                                                   'total_area', 'latitude',
                                                   'longitude',
                                                   'ceiling_height',
                                                   'flats_count',
                                                   'floors_total',
                                                   'building_age'])])),
                ('model',
                 <catboost.core.CatBoostRegressor object at 0x000001A0F1E62050>)])

In [13]:
# Оцениваем ошибку на тестовых данных
y_pred = pipeline.predict(X_test)
mape = mean_absolute_percentage_error(y_test, y_pred)
print(mape)

0.3985588563807358


In [14]:
# Сохраняем пайплайн в файл
os.makedirs('../services/ml_service/models', exist_ok=True)
with open("../services/ml_service/models/flats_prices_fitted_pipeline.pkl", 'wb') as fd:
    joblib.dump(pipeline, fd)

In [15]:
# Пробуем загрузить пайплайн из файла
pipeline = joblib.load('../services/ml_service/models/flats_prices_fitted_pipeline.pkl')
pipeline

Pipeline(steps=[('preproc_w_afr',
                 ColumnTransformer(n_jobs=-1, remainder='passthrough',
                                   transformers=[('bin_cat',
                                                  OneHotEncoder(drop='if_binary',
                                                                handle_unknown='ignore',
                                                                sparse_output=False),
                                                  []),
                                                 ('other_cat',
                                                  CatBoostEncoder(random_state=42),
                                                  []),
                                                 ('num_pipeline_w_afr',
                                                  Pipeline(steps=[('num_preproc_w_afr',
                                                                   ColumnTransformer(n_jobs=-1,
                                                                                     transformers=[('num_pol...
                                                                                                     'total_area',
                                                                                                     'latitude',
                                                                                                     'longitude',
                                                                                                     'ceiling_height',
                                                                                                     'flats_count',
                                                                                                     'floors_total',
                                                                                                     'building_age'])])),
                                                                  ('num_sc',
                                                                   StandardScaler())]),
                                                  ['floor', 'kitchen_area',
                                                   'living_area', 'rooms',
                                                   'total_area', 'latitude',
                                                   'longitude',
                                                   'ceiling_height',
                                                   'flats_count',
                                                   'floors_total',
                                                   'building_age'])])),
                ('model',
                 <catboost.core.CatBoostRegressor object at 0x000001A0F26DCB90>)])

Тестируем на произвольном наборе данных

In [16]:
# Словарь со входными параметрами
model_params = {
    'floor': 6,
    'kitchen_area': 8.5,
    'living_area': 30.0,
    'rooms': 2,
    'is_apartment': False,
    'total_area': 50.0,
    'building_age': 2024 - 1979,
    'building_type_int': 4,
    'latitude': 60.0,
    'longitude': 40.0,
    'ceiling_height': 2.5,
    'flats_count': 190,
    'floors_total': 12,
    'has_elevator': True
}

In [17]:
# Преобразуем параметры в датафрейм
model_params_df = pd.DataFrame(model_params, index=[0])

In [18]:
# Делаем инференс
pipeline.predict(model_params_df)[0]

8494489.99991846